## Ceneo Scraper

## Struktura opinii w serwisie ceneo.pl

|składowa|Selektor|zmienna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|.user-post__author-name|author|
|rekomendację|.user-post__author-recomendation >  em|recommend|
|liczbę gwiazdek|.user-post__score-count|stars|
|data wystawienia opinii|span.user-post__published > time::nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time::nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|
|treść opinii|.user-post__text|content|
|listę wad|div.review-feature__title--negatives~div.review-feature__item|cons|
|listę zalet|div.review-feature__title--positives~div.review-feature__col|pros|

1. Import Bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

2. Wczytanie kodu produktu  o którym mają zostać wczytane opinie


In [ ]:
product_id="104109736"
url=f'https://www.ceneo.pl/{product_id}#tab=reviews'

3. Wysłanie do serwera Ceneo.pl żądania dostępu do strony z opiniami o produkcie

In [ ]:
response=requests.get(url)
response.status_code

4. Przekształcenie kodu html w postaci tekstowej do obiektu reprezentującego strukturę DOM
5. Wydobycie ze struktry DOm znacznikow odpowiadających opiniom konsumentów

In [ ]:
page = BeautifulSoup(response.text,"html.parser")
opinions=page.select("div.js_product-review")
opinion=page.select_one("div.js_product-review")
print(type(page))
print(type(opinions))
print(type(opinion))



6. Wydobycie ze struktury znacznika odpowiadającego pojedynczej opinii jej składowych

In [ ]:
def extract(ancestor,selector, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select("selector")]
    if selector:
        if attribute:
            return ancestor.select_one(selector)[attribute].strip()
        return ancestor.select_one(selector).text.strip()
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

In [ ]:
single_opinion={
    "opinion_id": extract(opinion, None,"data-entry-id").strip()
    "author": extract(opinion,"span.user-post__author-name")
    "recommend": extract(opinion,".user-post__author-recomendation > em")
    "stars": extract(opinion,".user-post__score-count")
    "content": extract(opinion,"div.user-post__text")
    "cons": extract(opinion,"div.review-feature__title--negatives ~ div.review-feature__item", None, True)
    "pros": extract(opinion,"div.review-feature__title--positives ~ div.review-feature__item", None, True)
    "opinion_date": extract(opinion,"span.user-post__published > time::nth-child(1)","datetime")
    "purchase_date": extract(opinion,"span.user-post__published > time::nth-child(2)","datetime")
    "up_vote": extract(opinion,"button.vote-yes","data-total-vote")
    "down_vote": extract(opinion,"button.vote-no","data-total-vote")
}